In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import pandasql as psql
from pandasql import sqldf

In [2]:
# Define the function to perform T-Tests and calculate Cohen's d
def perform_t_tests(df, metrics, alpha=0.05):
    # Get unique values for 'area' and 'tenured'
    areas = df['area'].unique()
    tenured_statuses = df['tenured'].unique()
    
    # Function to interpret Cohen's d
    def interpret_cohen_d(d):
        if abs(d) < 0.2:
            return "Small"
        elif abs(d) < 0.5:
            return "Medium"
        else:
            return "Large"

    # Initialize a list to store results
    results = []

    for tenured in tenured_statuses:
        for area in areas:
            for metric in metrics:
                # Subset data based on 'area' and 'tenured'
                subset = df[(df['tenured'] == tenured) & (df['area'] == area)]
                # Separate data into 'HOME' and 'OFFICE' groups
                wfh_data = subset[subset['day_type'] == 'HOME'][metric]
                office_data = subset[subset['day_type'] == 'OFFICE'][metric]
                
                if len(wfh_data) > 0 and len(office_data) > 0:
                    # Perform T-Test
                    t_stat, p_value = ttest_ind(wfh_data, office_data, equal_var=False, nan_policy='omit')
                    
                    # Calculate means and sample sizes
                    wfh_mean = wfh_data.mean()
                    office_mean = office_data.mean()
                    wfh_std = wfh_data.std()
                    office_std = office_data.std()
                    n_wfh = len(wfh_data)
                    n_office = len(office_data)
                    
                    # Calculate pooled standard deviation
                    pooled_std = np.sqrt(((n_wfh - 1) * wfh_std**2 + (n_office - 1) * office_std**2) / (n_wfh + n_office - 2))
                    
                    # Calculate Cohen's d
                    cohen_d = (wfh_mean - office_mean) / pooled_std
                    cohen_d_interpretation = interpret_cohen_d(cohen_d)
                    
                    # Append results to the list
                    results.append({
                        'Tenured': tenured,
                        'Area': area,
                        'Metric': metric,
                        'WFH Mean': wfh_mean,
                        'Office Mean': office_mean,
                        'T-Statistic': t_stat,
                        'P-Value': round(p_value, 5),
                        'Cohen D': round(cohen_d, 5),
                        'Cohen D Interpretation': cohen_d_interpretation
                    })
        
        # Add combined area for the current tenured status
        for metric in metrics:
            combined_subset = df[df['tenured'] == tenured]
            wfh_data_combined = combined_subset[combined_subset['day_type'] == 'HOME'][metric]
            office_data_combined = combined_subset[combined_subset['day_type'] == 'OFFICE'][metric]
            
            if len(wfh_data_combined) > 0 and len(office_data_combined) > 0:
                # Perform T-Test
                t_stat_combined, p_value_combined = ttest_ind(wfh_data_combined, office_data_combined, equal_var=False, nan_policy='omit')
                
                # Calculate means and sample sizes
                wfh_mean_combined = wfh_data_combined.mean()
                office_mean_combined = office_data_combined.mean()
                wfh_std_combined = wfh_data_combined.std()
                office_std_combined = office_data_combined.std()
                n_wfh_combined = len(wfh_data_combined)
                n_office_combined = len(office_data_combined)
                
                # Calculate pooled standard deviation
                pooled_std_combined = np.sqrt(((n_wfh_combined - 1) * wfh_std_combined**2 + (n_office_combined - 1) * office_std_combined**2) / (n_wfh_combined + n_office_combined - 2))
                
                # Calculate Cohen's d
                cohen_d_combined = (wfh_mean_combined - office_mean_combined) / pooled_std_combined
                cohen_d_combined_interpretation = interpret_cohen_d(cohen_d_combined)
                
                # Append combined area results to the list
                results.append({
                    'Tenured': tenured,
                    'Area': 'All Areas Combined',
                    'Metric': metric,
                    'WFH Mean': wfh_mean_combined,
                    'Office Mean': office_mean_combined,
                    'T-Statistic': t_stat_combined,
                    'P-Value': round(p_value_combined, 5),
                    'Cohen D': round(cohen_d_combined, 5),
                    'Cohen D Interpretation': cohen_d_combined_interpretation
                })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Assuming 'result_df' is your DataFrame loaded with your data
metrics = ['avg_interaction_count', 'aht', 'avg_productivity']

# Set the alpha level (e.g., 0.05 for a 95% confidence level)
alpha = 0.05

# Perform the T-Tests and get the results
t_test_results = perform_t_tests(result_df, metrics)

print(t_test_results.head(5))

NameError: name 'result_df' is not defined

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from math import asin, sqrt

# Transcribe the data into a list of dictionaries
data = [
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Adherence", "WFH Proportion": 0.755},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Consults", "WFH Proportion": 0.153},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "OSAT", "WFH Proportion": 0.552},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Transfers", "WFH Proportion": 0.129},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Adherence", "WFH Proportion": 0.912},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Consults", "WFH Proportion": 0.206},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "OSAT", "WFH Proportion": 0.472},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Transfers", "WFH Proportion": 0.148},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Adherence", "WFH Proportion": 0.836},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Consults", "WFH Proportion": 0.178},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "OSAT", "WFH Proportion": 0.468},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Transfers", "WFH Proportion": 0.188},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Adherence", "WFH Proportion": 0.888},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Consults", "WFH Proportion": 0.104},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "OSAT", "WFH Proportion": 1.000},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Transfers", "WFH Proportion": 0.241},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Adherence", "WFH Proportion": 0.822},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Consults", "WFH Proportion": 0.266},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "OSAT", "WFH Proportion": 0.571},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Transfers", "WFH Proportion": 0.273},
]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Adding some hypothetical Office Proportion data
np.random.seed(42)  # For reproducibility
df["Office Proportion"] = np.random.rand(len(df))

# Set the alpha level (e.g., 0.05 for a 95% confidence level)
alpha = 0.05

# Function to interpret Cohen's h
def interpret_cohen_h(h):
    if abs(h) < 0.2:
        return "Small"
    elif abs(h) < 0.5:
        return "Medium"
    else:
        return "Large"

# Perform Z-tests for each row
results = []

for _, row in df.iterrows():
    count = np.array([row["WFH Proportion"] * row["Sample"], row["Office Proportion"] * row["Sample"]])
    nobs = np.array([row["Sample"], row["Sample"]])
    
    z_score, p_value = proportions_ztest(count, nobs)
    
    # Calculate Cohen's h
    cohen_h = 2 * (asin(sqrt(row["WFH Proportion"])) - asin(sqrt(row["Office Proportion"])))
    
    results.append({
        "Tenured": row["Tenured"],
        "Area Name": row["Area Name"],
        "Sample": row["Sample"],
        "Metric": row["Metric"],
        "WFH Proportion": row["WFH Proportion"],
        "Office Proportion": row["Office Proportion"],
        "Z-Statistic": z_score,
        "P-Value": p_value,
        "Cohen H": cohen_h,
        "Cohen H Interpretation": interpret_cohen_h(cohen_h),
        "Significant": p_value < alpha  # Check if the p-value is less than the alpha level
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Add combined area for each tenured status
combined_results = []

for tenured in df['Tenured'].unique():
    for metric in df['Metric'].unique():
        combined_subset = df[(df['Tenured'] == tenured) & (df['Metric'] == metric)]
        combined_sample = combined_subset['Sample'].sum()
        combined_wfh_proportion = (combined_subset['WFH Proportion'] * combined_subset['Sample']).sum() / combined_sample
        combined_office_proportion = (combined_subset['Office Proportion'] * combined_subset['Sample']).sum() / combined_sample
        
        count_combined = np.array([combined_wfh_proportion * combined_sample, combined_office_proportion * combined_sample])
        nobs_combined = np.array([combined_sample, combined_sample])
        
        z_score_combined, p_value_combined = proportions_ztest(count_combined, nobs_combined)
        
        # Calculate Cohen's h for combined data
        cohen_h_combined = 2 * (asin(sqrt(combined_wfh_proportion)) - asin(sqrt(combined_office_proportion)))
        
        combined_results.append({
            "Tenured": tenured,
            "Area Name": "COMBINED",
            "Sample": combined_sample,
            "Metric": metric,
            "WFH Proportion": combined_wfh_proportion,
            "Office Proportion": combined_office_proportion,
            "Z-Statistic": z_score_combined,
            "P-Value": p_value_combined,
            "Cohen H": cohen_h_combined,
            "Cohen H Interpretation": interpret_cohen_h(cohen_h_combined),
            "Significant": p_value_combined < alpha  # Check if the p-value is less than the alpha level
        })

# Convert combined results to a DataFrame and append to the original results
df_combined_results = pd.DataFrame(combined_results)
df_final_results = pd.concat([df_results, df_combined_results], ignore_index=True)

# Reorder the columns
df_final_results = df_final_results[["Tenured", "Area Name", "Sample", "Metric", "WFH Proportion", "Office Proportion", "Z-Statistic", "P-Value", "Cohen H Interpretation", "Significant"]]

print(df_final_results.head(5))

In [3]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from math import asin, sqrt

# Define WFH and Office days
wfh_days = ['Tuesday', 'Friday']
office_days = ['Monday', 'Wednesday', 'Thursday']

# Function to interpret Cohen's h
def interpret_cohen_h(h):
    if abs(h) < 0.2:
        return "Small"
    elif abs(h) < 0.5:
        return "Medium"
    else:
        return "Large"

# Function to get the overall difference in proportions by area and tenure
def area_tenure_z_tests(df, metrics_dict):
    area_names = df['area_name'].unique()
    tenured_statuses = df['tenured'].unique()
    results = []

    for tenured in tenured_statuses:
        for area in area_names:
            for metric, (num_col, den_col) in metrics_dict.items():
                subset = df[(df['tenured'] == tenured) & (df['area_name'] == area)]
                wfh_subset = subset[subset['day'].isin(wfh_days)]
                office_subset = subset[subset['day'].isin(office_days)]
                wfh_numerator = wfh_subset[num_col].sum()
                wfh_denominator = wfh_subset[den_col].sum()
                office_numerator = office_subset[num_col].sum()
                office_denominator = office_subset[den_col].sum()

                if wfh_denominator == 0 or office_denominator == 0:
                    continue

                count = np.array([wfh_numerator, office_numerator])
                observations = np.array([wfh_denominator, office_denominator])
                z_stat, p_value = proportions_ztest(count, observations)
                wfh_proportion = wfh_numerator / wfh_denominator if wfh_denominator else None
                office_proportion = office_numerator / office_denominator if office_denominator else None

                # Calculate Cohen's h
                cohen_h = 2 * (asin(sqrt(wfh_proportion)) - asin(sqrt(office_proportion)))
                cohen_h_interpretation = interpret_cohen_h(cohen_h)

                results.append({
                    'Tenured': tenured,
                    'Area Name': area,
                    'Metric': metric,
                    'WFH Proportion': wfh_proportion,
                    'Office Proportion': office_proportion,
                    'Z-Statistic': z_stat,
                    'P-Value': p_value.round(5),
                    'Cohen H': round(cohen_h, 5),
                    'Cohen H Interpretation': cohen_h_interpretation
                })

    return pd.DataFrame(results)

# Function to get the overall difference in proportions by tenure (area agnostic)
def combined_z_tests(df, metrics_dict):
    tenured_statuses = df['tenured'].unique()
    results = []

    for tenured in tenured_statuses:
        for metric, (num_col, den_col) in metrics_dict.items():
            subset = df[df['tenured'] == tenured]
            wfh_subset = subset[subset['day'].isin(wfh_days)]
            office_subset = subset[subset['day'].isin(office_days)]
            wfh_numerator = wfh_subset[num_col].sum()
            wfh_denominator = wfh_subset[den_col].sum()
            office_numerator = office_subset[num_col].sum()
            office_denominator = office_subset[den_col].sum()

            if wfh_denominator == 0 or office_denominator == 0:
                continue

            count = np.array([wfh_numerator, office_numerator])
            observations = np.array([wfh_denominator, office_denominator])
            z_stat, p_value = proportions_ztest(count, observations)
            wfh_proportion = wfh_numerator / wfh_denominator if wfh_denominator else None
            office_proportion = office_numerator / office_denominator if office_denominator else None

            # Calculate Cohen's h
            cohen_h = 2 * (asin(sqrt(wfh_proportion)) - asin(sqrt(office_proportion)))
            cohen_h_interpretation = interpret_cohen_h(cohen_h)

            results.append({
                'Tenured': tenured,
                'Area Name': "COMBINED",
                'Metric': metric,
                'WFH Proportion': wfh_proportion,
                'Office Proportion': office_proportion,
                'Z-Statistic': z_stat,
                'P-Value': p_value.round(5),
                'Cohen H': round(cohen_h, 5),
                'Cohen H Interpretation': cohen_h_interpretation
            })

    return pd.DataFrame(results)

# Test

metrics_dict = {
    'Adherence': ('adh_num', 'adh_den'),
    'OSAT': ('top_box', 'osat_count'),
    'Transfers': ('transfers', 'interaction_count'),
    'Consults': ('consults', 'interaction_count')
}

area_tenure_z_test_df = area_tenure_z_tests(result_df, metrics_dict)
combined_z_test_df = combined_z_tests(result_df, metrics_dict)

z_test = pd.concat([area_tenure_z_test_df, combined_z_test_df])

print(z_test.head(5))

NameError: name 'result_df' is not defined

In [4]:
import pandas as pd

# Data for the DataFrame
data = {
    'day_type': [
        'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE',
        'HOME', 'HOME', 'HOME', 'HOME', 'HOME',
        'OFFICE', 'HOME', 'OFFICE', 'HOME', 'OFFICE'
    ],
    'date': [
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03'
    ],
    'person_id': [
        'A003083', 'A003091', 'A003434', 'A003687', 'A003771', 
        'A004173', 'A004173', 'A004432', 'A005648', 'A007225', 
        'A008689', 'A009607', 'A009687', 'A015864', 'A017412'
    ],
    'area_name': [
        'High Net Worth', 'Chat', 'High Net Worth', 'Trader Service', 'High Net Worth', 
        'Core Service', 'Core Service', 'Trader Service', 'Trader Service', 'Chat', 
        'High Net Worth', 'Core Service', 'Core Service', 'High Net Worth', 'Core Service'
    ],
    'tenured': ['Yes'] * 15,
    'talk_time': [
        2353, 34945, 6693, 12467, 6444, 
        18897, 18897, 11115, 7036, 13667, 
        3786, 12718, 13221, 2695, 2200
    ],
    'wrap_time': [
        749, 0, 1402, 4207, 84, 
        3399, 3399, 1736, 4361, 0, 
        27, 2010, 663, 27, 749
    ],
    'interaction_count': [
        9, 51, 12, 26, 7, 
        33, 33, 14, 22, 18, 
        10, 15, 18, 9, 4
    ],
    'total_head_ct': [
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468
    ],
    'tenure_head_ct': [
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840
    ],
    'area_tenure_head_ct': [
        241, 209, 241, 326, 241, 
        764, 764, 326, 326, 209, 
        241, 764, 764, 241, 764
    ]
}

# Create the DataFrame
result_df = pd.DataFrame(data)

# Display the DataFrame
print(result_df)

   day_type        date person_id       area_name tenured  talk_time  \
0    OFFICE  2024-01-03   A003083  High Net Worth     Yes       2353   
1    OFFICE  2024-01-03   A003091            Chat     Yes      34945   
2    OFFICE  2024-01-03   A003434  High Net Worth     Yes       6693   
3    OFFICE  2024-01-03   A003687  Trader Service     Yes      12467   
4    OFFICE  2024-01-03   A003771  High Net Worth     Yes       6444   
5      HOME  2024-01-03   A004173    Core Service     Yes      18897   
6      HOME  2024-01-03   A004173    Core Service     Yes      18897   
7      HOME  2024-01-03   A004432  Trader Service     Yes      11115   
8      HOME  2024-01-03   A005648  Trader Service     Yes       7036   
9      HOME  2024-01-03   A007225            Chat     Yes      13667   
10   OFFICE  2024-01-03   A008689  High Net Worth     Yes       3786   
11     HOME  2024-01-03   A009607    Core Service     Yes      12718   
12   OFFICE  2024-01-03   A009687    Core Service     Yes      1

In [38]:
query = """
WITH CTE AS
(
SELECT 
  day_type
, area_name
, tenured
, area_tenure_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, tenured
, tenure_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4


UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, "COMBINED" AS tenured
, total_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4
)

SELECT
  HOME.area_name
, HOME.tenured
, HOME.head_count
, HOME.sample AS sample_home
, OFFICE.sample AS sample_office
, HOME.aht AS aht_home
, OFFICE.aht AS aht_office
FROM CTE HOME
INNER JOIN CTE OFFICE
    ON HOME.area_name = OFFICE.area_name
    AND HOME.tenured = OFFICE.tenured
    AND HOME.head_count = OFFICE.head_count
    AND OFFICE.day_type = 'OFFICE'
WHERE HOME.day_type = 'HOME'
ORDER BY HOME.area_name, HOME.tenured

"""

aht_df = sqldf(query)
print(aht_df.head(5))

        area_name   tenured  head_count  sample_home  sample_office  aht_home  \
0        COMBINED  COMBINED        2468          144            137  694.1458   
1        COMBINED       Yes        1840          144            137  694.1458   
2            Chat       Yes         209           18             51  759.2778   
3    Core Service       Yes         764           81             22  732.3457   
4  High Net Worth       Yes         241            9             38  302.4444   
5  Trader Service       Yes         326           36             26  673.5556   

   aht_office  
0    656.8613  
1    656.8613  
2    685.1961  
3    765.1364  
4    566.7895  
5    641.3077  


In [47]:
query = """
WITH CTE AS (

SELECT 
  day_type
, area_name
, tenured
, area_tenure_head_ct AS head_count
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
, COUNT(DISTINCT person_id || date) AS sample_size -- This calculates the sample size for each group
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, tenured
, tenure_head_ct AS head_count
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
, COUNT(DISTINCT person_id || date) AS sample_size -- This calculates the sample size for each group
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, "COMBINED" AS tenured
, total_head_ct AS head_count
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
, COUNT(DISTINCT person_id || date) AS sample_size -- This calculates the sample size for each group
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4
)

SELECT
  HOME.area_name
, HOME.tenured
, HOME.head_count
, HOME.sample_size AS sample_home
, OFFICE.sample_size AS sample_office
, HOME.aht AS aht_home
, OFFICE.aht AS aht_office
, HOME.sample_size AS sample_size_home  -- Sample size for 'HOME' days
, OFFICE.sample_size AS sample_size_office -- Sample size for 'OFFICE' days
FROM CTE HOME
INNER JOIN CTE OFFICE
    ON HOME.area_name = OFFICE.area_name
    AND HOME.tenured = OFFICE.tenured
    AND HOME.head_count = OFFICE.head_count
    AND OFFICE.day_type = 'OFFICE'
WHERE HOME.day_type = 'HOME'
ORDER BY HOME.area_name, HOME.tenured
"""

aht_df = sqldf(query)
print(aht_df.head(5))

        area_name   tenured  head_count  sample_home  sample_office  aht_home  \
0        COMBINED  COMBINED        2468            6              8  694.1458   
1        COMBINED       Yes        1840            6              8  694.1458   
2            Chat       Yes         209            1              1  759.2778   
3    Core Service       Yes         764            2              2  732.3457   
4  High Net Worth       Yes         241            1              4  302.4444   

   aht_office  sample_size_home  sample_size_office  
0    656.8613                 6                   8  
1    656.8613                 6                   8  
2    685.1961                 1                   1  
3    765.1364                 2                   2  
4    566.7895                 1                   4  


In [48]:
from scipy import stats

# Calculate the standard deviation for 'home' and 'office' AHTs
aht_df['std_home'] = aht_df['aht_home'] / np.sqrt(aht_df['sample_home'])
aht_df['std_office'] = aht_df['aht_office'] / np.sqrt(aht_df['sample_office'])

# Calculate the t-value
aht_df['t_value'] = (aht_df['aht_home'] - aht_df['aht_office']) / np.sqrt(
    (aht_df['std_home']**2 / aht_df['sample_home']) + 
    (aht_df['std_office']**2 / aht_df['sample_office'])
)

# Calculate the degrees of freedom using the Welch-Satterthwaite equation
aht_df['df'] = ((aht_df['std_home']**2 / aht_df['sample_home']) + 
                (aht_df['std_office']**2 / aht_df['sample_office']))**2 / (
                ((aht_df['std_home']**2 / aht_df['sample_home'])**2 / (aht_df['sample_home'] - 1)) + 
                ((aht_df['std_office']**2 / aht_df['sample_office'])**2 / (aht_df['sample_office'] - 1))
              )

# Calculate the p-value using the t-distribution survival function
aht_df['p_value'] = stats.t.sf(np.abs(aht_df['t_value']), aht_df['df']) * 2  # Two-tailed test

# Round the p-value to 3 decimal places
aht_df['p_value'] = aht_df['p_value'].round(3)


# Calculate pooled standard deviation for Cohen's d
pooled_std = np.sqrt(
    ((aht_df['sample_home'] - 1) * aht_df['std_home']**2 + 
     (aht_df['sample_office'] - 1) * aht_df['std_office']**2) / 
    (aht_df['sample_home'] + aht_df['sample_office'] - 2)
)

# Calculate Cohen's d
aht_df['cohen_d'] = (aht_df['aht_home'] - aht_df['aht_office']) / pooled_std

# Function to interpret Cohen's d
def interpret_cohen_d(d):
    if abs(d) < 0.2:
        return "Small"
    elif abs(d) < 0.5:
        return "Medium"
    else:
        return "Large"

# Apply the interpretation function to Cohen's d values
aht_df['effect_size'] = aht_df['cohen_d'].apply(interpret_cohen_d)


# Display the DataFrame with the new calculations
print(aht_df)

        area_name   tenured  head_count  sample_home  sample_office  aht_home  \
0        COMBINED  COMBINED        2468            6              8  694.1458   
1        COMBINED       Yes        1840            6              8  694.1458   
2            Chat       Yes         209            1              1  759.2778   
3    Core Service       Yes         764            2              2  732.3457   
4  High Net Worth       Yes         241            1              4  302.4444   
5  Trader Service       Yes         326            2              1  673.5556   

   aht_office  sample_size_home  sample_size_office    std_home  std_office  \
0    656.8613                 6                   8  283.383836  232.235540   
1    656.8613                 6                   8  283.383836  232.235540   
2    685.1961                 1                   1  759.277800  685.196100   
3    765.1364                 2                   2  517.846611  541.033137   
4    566.7895                 1      

In [14]:
import pandas as pd
import numpy as np

# Assuming aht_df has already been created from your previous steps
# Grouping by 'area_name', 'tenured', and 'day_type' to calculate variances

# Create a function to calculate variance
def calculate_variance(group):
    sample_mean = group['aht'].mean()
    sample_size = group['sample'].mean()  # We take mean as sample size for each subgroup
    variance = np.sum((group['aht'] - sample_mean)**2) / (sample_size - 1)
    return variance

# Separate the data into 'HOME' and 'OFFICE' groups
home_group = aht_df[aht_df['day_type'] == 'HOME']
office_group = aht_df[aht_df['day_type'] == 'OFFICE']

# Calculate variance for each group
home_variances = home_group.groupby(['area_name', 'tenured']).apply(calculate_variance).reset_index(name='variance')
office_variances = office_group.groupby(['area_name', 'tenured']).apply(calculate_variance).reset_index(name='variance')

# Merge the variances back to the original aht_df
home_variances = home_variances.rename(columns={'variance': 'home_variance'})
office_variances = office_variances.rename(columns={'variance': 'office_variance'})

# Now merging variances back with the main DataFrame
aht_variance_df = pd.merge(aht_df, home_variances, on=['area_name', 'tenured'], how='left')
aht_variance_df = pd.merge(aht_variance_df, office_variances, on=['area_name', 'tenured'], how='left')

# Display the DataFrame with calculated variances
print(aht_variance_df.head(20))

   day_type       area_name   tenured  sample       aht  home_variance  \
0      HOME        COMBINED  COMBINED    2468  694.1458            0.0   
1      HOME        COMBINED       Yes    1840  694.1458            0.0   
2      HOME            Chat       Yes     209  759.2778            0.0   
3      HOME    Core Service       Yes     764  732.3457            0.0   
4      HOME  High Net Worth       Yes     241  302.4444            0.0   
5      HOME  Trader Service       Yes     326  673.5556            0.0   
6    OFFICE        COMBINED  COMBINED    2468  656.8613            0.0   
7    OFFICE        COMBINED       Yes    1840  656.8613            0.0   
8    OFFICE            Chat       Yes     209  685.1961            0.0   
9    OFFICE    Core Service       Yes     764  765.1364            0.0   
10   OFFICE  High Net Worth       Yes     241  566.7895            0.0   
11   OFFICE  Trader Service       Yes     326  641.3077            0.0   

    office_variance  
0              

In [42]:
import pandas as pd
from pandasql import sqldf

# Data for the DataFrame (as you provided earlier)
data = {
    'day_type': [
        'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE', 'OFFICE',
        'HOME', 'HOME', 'HOME', 'HOME', 'HOME',
        'OFFICE', 'HOME', 'OFFICE', 'HOME', 'OFFICE'
    ],
    'date': [
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', 
        '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03', '2024-01-03'
    ],
    'person_id': [
        'A003083', 'A003091', 'A003434', 'A003687', 'A003771', 
        'A004173', 'A004173', 'A004432', 'A005648', 'A007225', 
        'A008689', 'A009607', 'A009687', 'A015864', 'A017412'
    ],
    'area_name': [
        'High Net Worth', 'Chat', 'High Net Worth', 'Trader Service', 'High Net Worth', 
        'Core Service', 'Core Service', 'Trader Service', 'Trader Service', 'Chat', 
        'High Net Worth', 'Core Service', 'Core Service', 'High Net Worth', 'Core Service'
    ],
    'tenured': ['Yes'] * 15,
    'talk_time': [
        2353, 34945, 6693, 12467, 6444, 
        18897, 18897, 11115, 7036, 13667, 
        3786, 12718, 13221, 2695, 2200
    ],
    'wrap_time': [
        749, 0, 1402, 4207, 84, 
        3399, 3399, 1736, 4361, 0, 
        27, 2010, 663, 27, 749
    ],
    'interaction_count': [
        9, 51, 12, 26, 7, 
        33, 33, 14, 22, 18, 
        10, 15, 18, 9, 4
    ],
    'total_head_ct': [
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468, 
        2468, 2468, 2468, 2468, 2468
    ],
    'tenure_head_ct': [
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840, 
        1840, 1840, 1840, 1840, 1840
    ],
    'area_tenure_head_ct': [
        241, 209, 241, 326, 241, 
        764, 764, 326, 326, 209, 
        241, 764, 764, 241, 764
    ]
}

# Create the DataFrame
result_df = pd.DataFrame(data)

# SQL query
query = """
WITH CTE AS
(
SELECT 
  day_type
, area_name
, tenured
, area_tenure_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, tenured
, tenure_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4

UNION

SELECT 
  day_type
, "COMBINED" AS area_name
, "COMBINED" AS tenured
, total_head_ct AS head_count
, SUM(interaction_count) AS sample
, ROUND(CAST(SUM(talk_time + wrap_time) AS FLOAT) / SUM(interaction_count), 4) AS aht
FROM result_df
WHERE interaction_count > 0
GROUP BY 1, 2, 3, 4
)

SELECT
  HOME.area_name
, HOME.tenured
, HOME.head_count
, HOME.sample AS sample_home
, OFFICE.sample AS sample_office
, HOME.aht AS aht_home
, OFFICE.aht AS aht_office
FROM CTE HOME
INNER JOIN CTE OFFICE
    ON HOME.area_name = OFFICE.area_name
    AND HOME.tenured = OFFICE.tenured
    AND HOME.head_count = OFFICE.head_count
    AND OFFICE.day_type = 'OFFICE'
WHERE HOME.day_type = 'HOME'
ORDER BY HOME.area_name, HOME.tenured
"""

# Execute the SQL query
aht_df = sqldf(query)

# Display the first 5 rows of the result
print(aht_df.head(5))

        area_name   tenured  head_count  sample_home  sample_office  aht_home  \
0        COMBINED  COMBINED        2468          144            137  694.1458   
1        COMBINED       Yes        1840          144            137  694.1458   
2            Chat       Yes         209           18             51  759.2778   
3    Core Service       Yes         764           81             22  732.3457   
4  High Net Worth       Yes         241            9             38  302.4444   

   aht_office  
0    656.8613  
1    656.8613  
2    685.1961  
3    765.1364  
4    566.7895  


In [44]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# Assuming you have your DataFrame `aht_df`

# Initialize lists to store results
t_values = []
p_values = []
effect_sizes = []

# Loop through each row in the DataFrame and perform the t-test
for index, row in aht_df.iterrows():
    # Generate random samples based on the given AHT and sample size
    np.random.seed(0)  # For reproducibility
    sample_home = np.random.normal(row['aht_home'], row['aht_home']/np.sqrt(row['sample_home']), int(row['sample_home']))
    sample_office = np.random.normal(row['aht_office'], row['aht_office']/np.sqrt(row['sample_office']), int(row['sample_office']))
    
    # Perform t-test
    t_stat, p_val = ttest_ind(sample_home, sample_office, equal_var=False)
    
    # Calculate Cohen's d
    pooled_std = np.sqrt(((len(sample_home) - 1) * np.std(sample_home, ddof=1) ** 2 + 
                          (len(sample_office) - 1) * np.std(sample_office, ddof=1) ** 2) / 
                         (len(sample_home) + len(sample_office) - 2))
    cohen_d = (np.mean(sample_home) - np.mean(sample_office)) / pooled_std
    
    # Interpret Cohen's d
    if abs(cohen_d) < 0.2:
        effect_size = "Small"
    elif abs(cohen_d) < 0.5:
        effect_size = "Medium"
    else:
        effect_size = "Large"
    
    # Store results
    t_values.append(t_stat)
    p_values.append(p_val)
    effect_sizes.append(effect_size)

# Add results to the DataFrame
aht_df['t_value'] = t_values
aht_df['p_value'] = np.round(p_values, 5)
aht_df['cohen_d'] = np.round(cohen_d, 5)
aht_df['effect_size'] = effect_sizes

# Display the updated DataFrame
print(aht_df)

        area_name   tenured  head_count  sample_home  sample_office  aht_home  \
0        COMBINED  COMBINED        2468          144            137  694.1458   
1        COMBINED       Yes        1840          144            137  694.1458   
2            Chat       Yes         209           18             51  759.2778   
3    Core Service       Yes         764           81             22  732.3457   
4  High Net Worth       Yes         241            9             38  302.4444   
5  Trader Service       Yes         326           36             26  673.5556   

   aht_office   t_value  p_value  cohen_d effect_size  
0    656.8613  7.429510  0.00000  0.83969       Large  
1    656.8613  7.429510  0.00000  0.83969       Large  
2    685.1961  5.679357  0.00001  0.83969       Large  
3    765.1364 -2.722768  0.01194  0.83969       Large  
4    566.7895 -4.749968  0.00051  0.83969       Large  
5    641.3077  3.311230  0.00162  0.83969       Large  
